In [21]:
pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector

# اتصال به پایگاه داده MySQL
connection = mysql.connector.connect(
  host="localhost",
  user="Bahareh",
  password="123456Bahareh"
)

cursor = connection.cursor()

# ایجاد پایگاه داده
cursor.execute("CREATE DATABASE IF NOT EXISTS store_management")

# استفاده از پایگاه داده جدید
cursor.execute("USE store_management")

# ایجاد جدول مشتریان
cursor.execute("CREATE TABLE IF NOT EXISTS customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), phone VARCHAR(255))")

# ایجاد جدول محصولات
cursor.execute("CREATE TABLE IF NOT EXISTS products (product_id INT AUTO_INCREMENT PRIMARY KEY, product_name VARCHAR(255), expiry_date DATE)")

# # ایجاد جدول خریدها
# cursor.execute("CREATE TABLE IF NOT EXISTS purchases (purchase_id INT AUTO_INCREMENT PRIMARY KEY, product_id INT, customer_id INT, purchase_date DATE, FOREIGN KEY (product_id) REFERENCES products(product_id), FOREIGN KEY (customer_id) REFERENCES customers(id))")

# ایجاد جدول خریدها
cursor.execute("CREATE TABLE IF NOT EXISTS purchases (purchase_id INT AUTO_INCREMENT PRIMARY KEY, product_id INT, customer_id INT, purchase_date DATE, purchase_order_id INT, FOREIGN KEY (product_id) REFERENCES products(product_id), FOREIGN KEY (customer_id) REFERENCES customers(id))")


# بستن اتصال
cursor.close()
connection.close()


In [6]:
import mysql.connector

# اتصال به پایگاه داده
connection = mysql.connector.connect(
  host="localhost",
  user="Bahareh",
  password="123456Bahareh"
)
cursor = connection.cursor()

# نمایش لیست دیتابیس‌ها
cursor.execute("SHOW DATABASES")
databases = cursor.fetchall()
print("لیست دیتابیس‌ها:")
for database in databases:
    print(database[0])

# بستن اتصال
cursor.close()
connection.close()


لیست دیتابیس‌ها:
db_store
information_schema
mft
mydatabase
mysql
performance_schema
store_db
sys


In [7]:
import mysql.connector



# اتصال به پایگاه داده
connection = mysql.connector.connect(
    host="localhost",
    user="Bahareh",
    password="123456Bahareh",
    database="mydatabase"  # نام دیتابیس مورد نظر خود را وارد کنید
)
cursor = connection.cursor()

# نمایش لیست جداول
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print("لیست جداول:")
for table in tables:
    print(table[0])

# بستن اتصال
cursor.close()
connection.close()


لیست جداول:
customers
products
purchases


In [8]:
import mysql.connector

def add_customer_to_database(name, customer_id, phone):
    try:
        # برقراری اتصال به دیتابیس
        connection = mysql.connector.connect(
            host="localhost",
            user="Bahareh",
            password="123456Bahareh",
            database="mydatabase"
        )

        # ایجاد cursor برای اجرای کوئری‌ها
        cursor = connection.cursor()

        # اجرای کوئری برای افزودن مشتری به جدول
        query = "INSERT INTO customers (name, id, phone) VALUES (%s, %s, %s)"
        cursor.execute(query, (name, customer_id, phone))

        # ذخیره تغییرات و بستن اتصال
        connection.commit()
        connection.close()

        print("مشتری با موفقیت به جدول اضافه شد.")
    except mysql.connector.Error as err:
        print(f"خطا در افزودن مشتری به دیتابیس: {err}")

# اضافه کردن مشتری با اطلاعات مشخص به دیتابیس
add_customer_to_database("بهاره", 1, "09055229672")


خطا در افزودن مشتری به دیتابیس: 1062 (23000): Duplicate entry '1' for key 'customers.PRIMARY'


In [3]:
pip install pyqt5


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from PyQt5.QtWidgets import QWidget
import mysql.connector
from PyQt5.QtWidgets import QApplication, QMainWindow, QAction, QDialog, QVBoxLayout, QLabel, QLineEdit, QPushButton, \
    QTableWidget, QTableWidgetItem, QMessageBox
import sys



class ProductForm(QDialog):
    def __init__(self, connection):
        super().__init__()
        self.connection = connection
        self.setWindowTitle("فرم افزودن محصول")
        self.layout = QVBoxLayout()

        self.name_label = QLabel("نام محصول:")
        self.name_entry = QLineEdit()
        self.layout.addWidget(self.name_label)
        self.layout.addWidget(self.name_entry)

        self.id_label = QLabel("آیدی:")
        self.id_entry = QLineEdit()
        self.layout.addWidget(self.id_label)
        self.layout.addWidget(self.id_entry)

        self.expiry_date_label = QLabel("تاریخ انقضا:")
        self.expiry_date_entry = QLineEdit()
        self.layout.addWidget(self.expiry_date_label)
        self.layout.addWidget(self.expiry_date_entry)

        # اضافه کردن فیلد قیمت محصول
        self.price_label = QLabel("قیمت:")
        self.price_entry = QLineEdit()
        self.layout.addWidget(self.price_label)
        self.layout.addWidget(self.price_entry)

        self.add_button = QPushButton("افزودن")
        self.add_button.clicked.connect(self.add_product)
        self.layout.addWidget(self.add_button)

        self.setLayout(self.layout)

    def add_product(self):
        name = self.name_entry.text()
        product_id = self.id_entry.text()
        expiry_date = self.expiry_date_entry.text()
        price = self.price_entry.text()  # دریافت قیمت محصول

        try:
            cursor = self.connection.cursor()
            # افزودن فیلد قیمت به دستور SQL
            cursor.execute("INSERT INTO products (product_name, product_id, expiry_date, price) VALUES (%s, %s, %s, %s)", (name, product_id, expiry_date, price))
            self.connection.commit()
            QMessageBox.information(self, "موفقیت آمیز", "محصول با موفقیت به جدول اضافه شد")
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در افزودن محصول به دیتابیس: {err}")

class EditProductForm(QDialog):
    def __init__(self, connection, product_id):
        super().__init__()
        self.connection = connection
        self.product_id = product_id
        self.setWindowTitle("ویرایش محصول")
        self.layout = QVBoxLayout()

        self.name_label = QLabel("نام محصول:")
        self.name_entry = QLineEdit()
        self.layout.addWidget(self.name_label)
        self.layout.addWidget(self.name_entry)

        self.expiry_date_label = QLabel("تاریخ انقضا:")
        self.expiry_date_entry = QLineEdit()
        self.layout.addWidget(self.expiry_date_label)
        self.layout.addWidget(self.expiry_date_entry)

        # اضافه کردن فیلد قیمت محصول
        self.price_label = QLabel("قیمت:")
        self.price_entry = QLineEdit()
        self.layout.addWidget(self.price_label)
        self.layout.addWidget(self.price_entry)

        self.submit_button = QPushButton("ثبت تغییرات")
        self.submit_button.clicked.connect(self.edit_product)
        self.layout.addWidget(self.submit_button)

        self.setLayout(self.layout)

        self.load_product_data()

    def load_product_data(self):
        try:
            cursor = self.connection.cursor()
            cursor.execute("SELECT * FROM products WHERE product_id = %s", (self.product_id,))
            product = cursor.fetchone()
            self.name_entry.setText(product[1])
            self.expiry_date_entry.setText(product[2].strftime("%Y-%m-%d"))
            # نمایش قیمت محصول در فیلد قیمت
            self.price_entry.setText(str(product[3]))
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در بارگیری اطلاعات محصول: {err}")

    def edit_product(self):
        name = self.name_entry.text()
        expiry_date = self.expiry_date_entry.text()
        price = self.price_entry.text()  # دریافت قیمت جدید محصول

        try:
            cursor = self.connection.cursor()
            cursor.execute("UPDATE products SET product_name = %s, expiry_date = %s, price = %s WHERE product_id = %s",
                           (name, expiry_date, price, self.product_id))
            self.connection.commit()
            QMessageBox.information(self, "موفقیت آمیز", "تغییرات با موفقیت ذخیره شد")
            self.close()
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در ویرایش محصول: {err}")


class CustomerForm(QDialog):
    def __init__(self, connection):
        super().__init__()
        self.connection = connection
        self.setWindowTitle("فرم اضافه کردن مشتری")
        self.layout = QVBoxLayout()

        self.name_label = QLabel("نام مشتری:")
        self.name_entry = QLineEdit()
        self.layout.addWidget(self.name_label)
        self.layout.addWidget(self.name_entry)

        self.id_label = QLabel("آیدی مشتری:")
        self.id_entry = QLineEdit()
        self.layout.addWidget(self.id_label)
        self.layout.addWidget(self.id_entry)

        self.phone_label = QLabel("شماره تماس:")
        self.phone_entry = QLineEdit()
        self.layout.addWidget(self.phone_label)
        self.layout.addWidget(self.phone_entry)

        self.submit_button = QPushButton("ثبت")
        self.submit_button.clicked.connect(self.add_customer)
        self.layout.addWidget(self.submit_button)

        self.setLayout(self.layout)

    def add_customer(self):
        name = self.name_entry.text()
        customer_id = self.id_entry.text()
        phone = self.phone_entry.text()

        try:
            cursor = self.connection.cursor()
            cursor.execute("INSERT INTO customers (name, id, phone) VALUES (%s, %s, %s)", (name, customer_id, phone))
            self.connection.commit()
            QMessageBox.information(self, "موفقیت آمیز", "اطلاعات مشتری با موفقیت ذخیره شد")
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در ثبت اطلاعات مشتری: {err}")


class PurchaseForm(QDialog):
    def __init__(self, connection):
        super().__init__()
        self.connection = connection
        self.setWindowTitle("فرم خرید محصول")
        self.layout = QVBoxLayout()

        self.product_id_label = QLabel("آیدی محصول:")
        self.product_id_entry = QLineEdit()
        self.layout.addWidget(self.product_id_label)
        self.layout.addWidget(self.product_id_entry)

        self.customer_id_label = QLabel("آیدی مشتری:")
        self.customer_id_entry = QLineEdit()
        self.layout.addWidget(self.customer_id_label)
        self.layout.addWidget(self.customer_id_entry)

        self.purchase_date_label = QLabel("تاریخ خرید:")
        self.purchase_date_entry = QLineEdit()
        self.layout.addWidget(self.purchase_date_label)
        self.layout.addWidget(self.purchase_date_entry)

        self.purchase_button = QPushButton("خرید")
        self.purchase_button.clicked.connect(self.purchase_product)
        self.layout.addWidget(self.purchase_button)

        self.setLayout(self.layout)

    def purchase_product(self):
        product_id = self.product_id_entry.text()
        customer_id = self.customer_id_entry.text()
        purchase_date = self.purchase_date_entry.text()

        try:
            cursor = self.connection.cursor()
            cursor.execute("INSERT INTO purchases (product_id, customer_id, purchase_date) VALUES (%s, %s, %s)",
                           (product_id, customer_id, purchase_date))
            self.connection.commit()
            QMessageBox.information(self, "موفقیت آمیز", "خرید با موفقیت ثبت شد")
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در ثبت خرید در دیتابیس: {err}")

class MainWindow(QMainWindow):
    def __init__(self, connection):
        super().__init__()
        self.connection = connection
        self.setWindowTitle("برنامه فروشگاه زنجیره‌ای")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QVBoxLayout()

        self.products_table = QTableWidget()
        self.products_table.setColumnCount(4)  # تعداد ستون‌ها را به 4 تغییر دهید
        self.products_table.setHorizontalHeaderLabels(["آيدي", "نام محصول", "تاریخ انقضا", "قیمت"])  # ستون مربوط به قیمت را اضافه کنید
        self.central_widget.addWidget(self.products_table)

        self.customers_table = QTableWidget()
        self.customers_table.setColumnCount(3)
        self.customers_table.setHorizontalHeaderLabels(["آيدي", "نام مشتری", "شماره تماس"])
        self.central_widget.addWidget(self.customers_table)

        self.purchases_table = QTableWidget()
        self.purchases_table.setColumnCount(4)
        self.purchases_table.setHorizontalHeaderLabels(["آيدي خريد", "آیدی محصول", "آیدی مشتري", "تاريخ خريد"])
        self.central_widget.addWidget(self.purchases_table)

        menu_bar = self.menuBar()
        product_menu = menu_bar.addMenu("محصول")
        customer_menu = menu_bar.addMenu("مشتری")
        purchase_menu = menu_bar.addMenu("خرید")

        add_product_action = QAction("ثبت محصول", self)
        add_product_action.triggered.connect(self.open_add_product_form)
        product_menu.addAction(add_product_action)
        delete_product_action = QAction("حذف محصول", self)
        delete_product_action.triggered.connect(self.delete_product)
        product_menu.addAction(delete_product_action)
        edit_product_action = QAction("ویرایش محصول", self)
        edit_product_action.triggered.connect(self.open_edit_product_form)
        product_menu.addAction(edit_product_action)

        add_customer_action = QAction("ثبت مشتری", self)
        add_customer_action.triggered.connect(self.open_add_customer_form)
        customer_menu.addAction(add_customer_action)
        delete_customer_action = QAction("حذف مشتری", self)
        delete_customer_action.triggered.connect(self.delete_customer)
        customer_menu.addAction(delete_customer_action)
        edit_customer_action = QAction("ویرایش مشتری", self)
        edit_customer_action.triggered.connect(self.open_edit_customer_form)
        customer_menu.addAction(edit_customer_action)

        purchase_action = QAction("ثبت خرید", self)
        purchase_action.triggered.connect(self.open_purchase_form)
        purchase_menu.addAction(purchase_action)
        delete_purchase_action = QAction("حذف خرید", self)
        delete_purchase_action.triggered.connect(self.delete_purchase)
        purchase_menu.addAction(delete_purchase_action)
        edit_purchase_action = QAction("ویرایش خرید", self)
        edit_purchase_action.triggered.connect(self.open_edit_purchase_form)
        purchase_menu.addAction(edit_purchase_action)

        self.load_data()

        central_widget = QWidget()
        central_widget.setLayout(self.central_widget)
        self.setCentralWidget(central_widget)

    def load_data(self):
        try:
            cursor = self.connection.cursor()

            cursor.execute("SELECT product_id, product_name, expiry_date, price FROM products")  # ستون قیمت را از دستور SQL انتخاب کنید
            products = cursor.fetchall()
            self.products_table.setRowCount(len(products))
            for i, product in enumerate(products):
                for j, data in enumerate(product):
                    self.products_table.setItem(i, j, QTableWidgetItem(str(data)))

            cursor.execute("SELECT * FROM customers")
            customers = cursor.fetchall()
            self.customers_table.setRowCount(len(customers))
            for i, customer in enumerate(customers):
                for j, data in enumerate(customer):
                    self.customers_table.setItem(i, j, QTableWidgetItem(str(data)))

            cursor.execute("SELECT * FROM purchases")
            purchases = cursor.fetchall()
            self.purchases_table.setRowCount(len(purchases))
            for i, purchase in enumerate(purchases):
                for j, data in enumerate(purchase):
                    self.purchases_table.setItem(i, j, QTableWidgetItem(str(data)))
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در بارگیری اطلاعات: {err}")

    def open_add_product_form(self):
        add_product_form = ProductForm(self.connection)
        add_product_form.exec_()
        self.load_data()

    def open_add_customer_form(self):
        add_customer_form = CustomerForm(self.connection)
        add_customer_form.exec_()
        self.load_data()

    def open_purchase_form(self):
        purchase_form = PurchaseForm(self.connection)
        purchase_form.exec_()
        self.load_data()

    def delete_product(self):
        selected_rows = self.products_table.selectionModel().selectedRows()
        if selected_rows:
            reply = QMessageBox.question(self, 'تأیید حذف', 'آیا از حذف محصول مطمئن هستید؟',
                                         QMessageBox.Yes | QMessageBox.No)
            if reply == QMessageBox.Yes:
                for row in selected_rows:
                    product_id = self.products_table.item(row.row(), 0).text()  # شناسه محصول
                    try:
                        cursor = self.connection.cursor()
                        cursor.execute("DELETE FROM products WHERE product_id = %s", (product_id,))
                        self.connection.commit()
                    except mysql.connector.Error as err:
                        QMessageBox.critical(self, "خطا", f"خطا در حذف محصول: {err}")
                self.load_data()
        else:
            QMessageBox.warning(self, "هشدار", "لطفاً یک محصول را انتخاب کنید")

    def delete_customer(self):
        selected_rows = self.customers_table.selectionModel().selectedRows()
        if selected_rows:
            reply = QMessageBox.question(self, 'تأیید حذف', 'آیا از حذف مشتری مطمئن هستید؟',
                                         QMessageBox.Yes | QMessageBox.No)
            if reply == QMessageBox.Yes:
                for row in selected_rows:
                    customer_id = self.customers_table.item(row.row(), 0).text()  # شناسه مشتری
                    try:
                        cursor = self.connection.cursor()
                        cursor.execute("DELETE FROM customers WHERE id = %s", (customer_id,))
                        self.connection.commit()
                    except mysql.connector.Error as err:
                        QMessageBox.critical(self, "خطا", f"خطا در حذف مشتری: {err}")
                self.load_data()
        else:
            QMessageBox.warning(self, "هشدار", "لطفاً یک مشتری را انتخاب کنید")

    def delete_purchase(self):
        selected_rows = self.purchases_table.selectionModel().selectedRows()
        if selected_rows:
            reply = QMessageBox.question(self, 'تأیید حذف', 'آیا از حذف خرید مطمئن هستید؟',
                                         QMessageBox.Yes | QMessageBox.No)
            if reply == QMessageBox.Yes:
                for row in selected_rows:
                    purchase_id = self.purchases_table.item(row.row(), 0).text()
                    try:
                        cursor = self.connection.cursor()
                        cursor.execute("DELETE FROM purchases WHERE purchase_id = %s", (purchase_id,))
                        self.connection.commit()
                    except mysql.connector.Error as err:
                        QMessageBox.critical(self, "خطا", f"خطا در حذف خرید: {err}")
                self.load_data()
        else:
            QMessageBox.warning(self, "هشدار", "لطفاً یک خرید را انتخاب کنید")

    def open_edit_product_form(self):
        selected_rows = self.products_table.selectionModel().selectedRows()
        if selected_rows:
            product_id = self.products_table.item(selected_rows[0].row(), 0).text()
            edit_product_form = EditProductForm(self.connection, product_id)
            edit_product_form.exec_()
            self.load_data()
        else:
            QMessageBox.warning(self, "هشدار", "لطفاً یک محصول را انتخاب کنید")

    def open_edit_customer_form(self):
        selected_rows = self.customers_table.selectionModel().selectedRows()
        if selected_rows:
            customer_id = self.customers_table.item(selected_rows[0].row(), 0).text()
            edit_customer_form = EditCustomerForm(self.connection, customer_id)
            edit_customer_form.exec_()
            self.load_data()
        else:
            QMessageBox.warning(self, "هشدار", "لطفاً یک مشتری را انتخاب کنید")

    def open_edit_purchase_form(self):
        selected_rows = self.purchases_table.selectionModel().selectedRows()
        if selected_rows:
            purchase_id = self.purchases_table.item(selected_rows[0].row(), 0).text()
            edit_purchase_form = EditPurchaseForm(self.connection, purchase_id)
            edit_purchase_form.exec_()
            self.load_data()
        else:
            QMessageBox.warning(self, "هشدار", "لطفاً یک خرید را انتخاب کنید")

class EditCustomerForm(QDialog):
    def __init__(self, connection, customer_id):
        super().__init__()
        self.connection = connection
        self.customer_id = customer_id
        self.setWindowTitle("ویرایش مشتری")
        self.layout = QVBoxLayout()

        self.name_label = QLabel("نام مشتری:")
        self.name_entry = QLineEdit()
        self.layout.addWidget(self.name_label)
        self.layout.addWidget(self.name_entry)

        self.phone_label = QLabel("شماره تماس:")
        self.phone_entry = QLineEdit()
        self.layout.addWidget(self.phone_label)
        self.layout.addWidget(self.phone_entry)

        self.submit_button = QPushButton("ثبت تغییرات")
        self.submit_button.clicked.connect(self.edit_customer)
        self.layout.addWidget(self.submit_button)

        self.setLayout(self.layout)

        self.load_customer_data()

    def load_customer_data(self):
        try:
            cursor = self.connection.cursor()
            cursor.execute("SELECT * FROM customers WHERE id = %s", (self.customer_id,))
            customer = cursor.fetchone()
            self.name_entry.setText(str(customer[1]))
            self.phone_entry.setText(str(customer[2]))
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در بارگیری اطلاعات مشتری: {err}")

    def edit_customer(self):
        name = self.name_entry.text()
        phone = self.phone_entry.text()

        try:
            cursor = self.connection.cursor()
            cursor.execute("UPDATE customers SET name = %s, phone = %s WHERE id = %s",
                           (name, phone, self.customer_id))
            self.connection.commit()
            QMessageBox.information(self, "موفقیت آمیز", "تغییرات با موفقیت ذخیره شد")
            self.close()
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در ویرایش مشتری: {err}")


class EditPurchaseForm(QDialog):
    def __init__(self, connection, purchase_id):
        super().__init__()
        self.connection = connection
        self.purchase_id = purchase_id
        self.setWindowTitle("ویرایش خرید")
        self.layout = QVBoxLayout()

        self.product_id_label = QLabel("آیدی محصول:")
        self.product_id_entry = QLineEdit()
        self.layout.addWidget(self.product_id_label)
        self.layout.addWidget(self.product_id_entry)

        self.customer_id_label = QLabel("آیدی مشتری:")
        self.customer_id_entry = QLineEdit()
        self.layout.addWidget(self.customer_id_label)
        self.layout.addWidget(self.customer_id_entry)

        self.purchase_date_label = QLabel("تاریخ خرید:")
        self.purchase_date_entry = QLineEdit()
        self.layout.addWidget(self.purchase_date_label)
        self.layout.addWidget(self.purchase_date_entry)

        self.submit_button = QPushButton("ثبت تغییرات")
        self.submit_button.clicked.connect(self.edit_purchase)
        self.layout.addWidget(self.submit_button)

        self.setLayout(self.layout)

        self.load_purchase_data()

    def load_purchase_data(self):
        try:
            cursor = self.connection.cursor()
            cursor.execute("SELECT * FROM purchases WHERE purchase_id = %s", (self.purchase_id,))
            purchase = cursor.fetchone()
            self.product_id_entry.setText(str(purchase[1]))
            self.customer_id_entry.setText(str(purchase[2]))
            self.purchase_date_entry.setText(str(purchase[3]))
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در بارگیری اطلاعات خرید: {err}")

    def edit_purchase(self):
        product_id = self.product_id_entry.text()
        customer_id = self.customer_id_entry.text()
        purchase_date = self.purchase_date_entry.text()

        try:
            cursor = self.connection.cursor()
            cursor.execute(
                "UPDATE purchases SET product_id = %s, customer_id = %s, purchase_date = %s WHERE purchase_id = %s",
                (product_id, customer_id, purchase_date, self.purchase_id))
            self.connection.commit()
            QMessageBox.information(self, "موفقیت آمیز", "تغییرات با موفقیت ذخیره شد")
            self.close()
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "خطا", f"خطا در ویرایش خرید: {err}")


def create_database_schema(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS mydba")
        cursor.execute("USE mydba")
        cursor.execute("""CREATE TABLE IF NOT EXISTS products (
                                    product_id INT AUTO_INCREMENT PRIMARY KEY,
                                    product_name VARCHAR(255),
                                    expiry_date DATE,
                                    price DECIMAL(10, 2) DEFAULT 0
                                    )""")

        cursor.execute("""CREATE TABLE IF NOT EXISTS customers (
                            id INT AUTO_INCREMENT PRIMARY KEY,
                            name VARCHAR(255),
                            phone VARCHAR(15)
                            )""")
        cursor.execute("""CREATE TABLE IF NOT EXISTS purchases (
                            purchase_id INT AUTO_INCREMENT PRIMARY KEY,
                            product_id INT,
                            customer_id INT,
                            purchase_date DATE,
                            FOREIGN KEY (product_id) REFERENCES products(product_id),
                            FOREIGN KEY (customer_id) REFERENCES customers(id)
                            )""")
        connection.commit()
    except mysql.connector.Error as err:
        print(f"Error: {err}")


def main():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="Bahareh",
            password="123456Bahareh"  # Enter your MySQL password here
        )
        create_database_schema(connection)
        app = QApplication(sys.argv)
        window = MainWindow(connection)
        window.show()
        sys.exit(app.exec_())
    except mysql.connector.Error as err:
        print(f"Error: {err}")

if __name__ == "__main__":
    main()

SystemExit: 0

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
